# Figure数据结构 #

## 底层结构的查看 ##
通过 **print(fig)** 或在 **jupyterlab** 中使用 **fig.show('json')** 渲染器，可以查看底层的数据结构，对任何plotly.graph_objects.Figure对象

In [1]:
import plotly.express as px

In [4]:
fig = px.line(x=['a', 'b', 'c'], y=[1, 3, 2], title='sample figure')
print(fig)
fig.show()

Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['a', 'b', 'c'], dtype=object),
              'xaxis': 'x',
              'y': array([1, 3, 2], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'sample figure'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})


现在所看到的的是以json格式呈现的底层结构:
* 整个为一个Figure对象，包含data和layout两层
    * data层属性：
        * **hovertemplate** 是一个包含宏的html文件，这里来看存了x和y的信息
        * **legendgroup** 储存了legend
        * **line**储存了线的性质，包含color和dash
        * **marker**记号，symbol属性为circle
        * **mode**模式，为lines
        * **name**名字，无
        * **orientation**未知
        * **showlegend**是否显示legend，否
        * **type**类型为scatter
        * **x** x标签，['a', 'b', 'c']
        * **xaxis** x轴，标签为x
        * **y** y标签，[1, 3, 2]
        * **yaxis** y轴，标签为y
    * layout层属性：
        * **legend** legend设置
        * **template** 未知
        * **title**标题设置，为文本，内容为“sample figure”
        * **xaxis** x轴设置，目标y，区域[0.0, 1.0]，标题为文本：x
        * **yaxis** y轴设置，目标x，区域[0.0, 1.0]，标题为文本：y

## 在 **Dash**中得到figure结构

略。详见官网[dash文件](https://python-docs-dash-snippets.herokuapp.com/assets/figure-structure.py)

## Figure 是属性树

Figure表示为具有称为“属性”的命名节点的树，树的节点有三个顶级属性：**data，layout，frames**
属性是一种“路径”,表示为“父类.属性.下一级属性”。如果父类是一个list而不是dict，那么要插入“[]”，例如“parents[].attribute”。注意“父类”表示的是上下级的关系，是相对的，上级的所有属性都可称为父类。
顶级属性定义了一个名为“traces”的类型对象列表，其架构依赖于类型，比如“data[type=scatter].name”。
plotly.graph_objects模块包含一个自动生成的Python类层次结构，这些类表示figure schema中的non-leaf属性，并提供了Python API在操作plotly.graph_objects.Figure 对象时，属性可直接用Python对象属性设置，如fig.layout.title.font.family="Open Sans"或者用update方法和”magic underscores“，如fig.update_layout(title_font_family="Open Sans"。
在构建图形时，不需要填充每个对象的每个属性，对未指定属性会计算默认值。优先级：指定>默认。

## 顶级**data**属性

第一个是**data**，它的值必须是称为“trace”的dict列表。
* 每个trace都有超过40种可能的类型中的一种（scatter,bar,pie,surface,choropleth等等），并在一个figure中表示一组相关的graphical marks。每个trace必有一个类型属性定义其他允许的属性。
* 每个trace绘制在一个单独的子图上，该子图类型必须与trace的类型兼容，或者是它自己的子图。
* Traces除了pie和funnelarea traces，可能只有一个legend条目。
* 某些trace类型支持连续的颜色，并带有一个相关的颜色条，可以通过trace中的属性或layout中的属性来控制它。

## 顶级**layout**属性

第二个是layout，其值在文本中称为“布局”，并且必须是一个dict，其中包含控制figure中非数据相关部分的定位和配置的属性，例如：
* Dimensions和margins，定义“纸面坐标“的边界
* Figure-wide defaults：包括 templates, fonts, colors, hover-label 和 modebar defaults
* Title and legend， 可以在容器和（或）纸坐标中定位
* Color axes and associated color bars，颜色轴和相关的颜色条（可在纸坐标中定位）
* 可以绘制多个迹线并将其放在纸坐标中的各种类型的subplots
    * xaxis, yaxis, xaxis2, yaxis3 etc:X和Y笛卡尔轴，其相交是笛卡尔子图
    * scene, scene2, scene3 etc: 3D场景子图
    * ternary, ternary2, ternary3, polar, polar2, polar3, geo, geo2, geo3, mapbox, mapbox2, mabox3, smith, smith2 etc: Ternary，Polar，Geo，Geo，Mapbox或Smith子图
*可以在纸坐标或与2D笛卡尔子图相关的数据坐标中定位的非数据标记
    * annotations:带有或没有箭头的文字注释
    * shapes:线，矩形，椭圆形或开放或封闭的路径
    * images: 背景或装饰图像
* 可以在纸坐标中定位的控件，并且可以触发plotly.js功能
    * updatemenus: 单按钮，切换和下拉菜单
    * sliders: 滑块控件

## 顶级**frames**属性

第三个是frames，其值必须是在动画图中定义顺序帧的dict列表。每个frame都包含其自己的数据属性以及其他参数。Animations通常是通过layout.slider和/或layout.updatemenus中定义的控件触发和控制的。

## config 对象

在渲染时间，还可以控制某些不被视为figure proper的figure行为，如“modebar”的行为，以及figure与鼠标动作如滚动等的关系。包含这些选项的对象称为config，并具有自己的documentation page。它在python中被视为plotly.graph_objects.figure对象的.show（）方法的config关键字参数。

## 用纸，容器坐标或轴域坐标定位
在指定X或Y的图支持定位属性的布局中配置的各种图形组件，其值可以在“纸坐标”中指定（有时称为“绘图分数”或“归一化坐标”）。示例包括layout.xaxis.domain或laygend.legend.x或layout.annotation[].x。

在纸坐标中定位不是绝对像素术语进行的，而是用相对于用原点（0,0）定义的坐标系（layout.margin.l，layout.margin.b）和一个点（1，1）在（layout.width-layout.margin.r，layout.height-layout.margin.t）（注意：layout.margin值是像素值，即layout.width and Layout.height）。允许纸坐标值小于0或大于1，并参考图边缘内的区域。

要将对象放置在“纸”坐标中，相应的轴引用设置为“纸”。例如，shape的xref属性将设置为“纸”，以便形状的X值指其在纸坐标中的位置。

请注意，layout.margin属性的内容默认是根据某些项目的位置和尺寸（例如title或legend）计算的，并且当设置Layout.xaxis.automargin=True时，可能取决于tick标签的位置和尺寸。这具有自动增加边距值的影响，因此在（0,0）和（1,1）点之间定义的物理面积缩小了。将某些项目定位在纸张坐标小于0或大于1的纸坐标也将触发这种行为。但是，layout.width和layout.height被视为已知，因此数字永远不会根据其内容而生长或收缩。

figure 的title可以使用“容器坐标”定位，它有（0,0）和（1,1）分别锚定在图的底部和最高右右，因此与layout.margin值无关。

此外，可以将相对于轴的域相对于轴的域而放置shapes, annotations, 和images，例如，x值为0.5，将对象沿x轴中间放置，而与layout.xaxis.domain属性无关。可以通过将“domain”添加到对象的轴引用属性中的轴引用中来指定此行为。例如，为形状设置yref ='y2 domani'将指名为y2的轴的长度和位置。

## 2D笛卡尔跟踪类型和子图

最常用的子图是二维笛卡尔子图。与这些子图兼容的traces支持Xaxis和Yaxis属性，其值必须指图的布局部分中的相应对象。例如，如果xaxis =“x”，而yaxis =“y”（这是默认值），则该trace是在layout.xaxis和layout.yaxis的轴相交的子图上绘制的，但是如果xaxis =“x2”和yaxis =“y3”，那么trace是在layout.xaxis2和layout.yaxis3下配置的轴的交点上绘制的。请注意，诸如layout.xaxis和layout.xaxis2等的属性不必明确定义，在这种情况下，将推断默认值。可以在同一子图上绘制不同类型的多个trace。

X-和Y-AXES支持类型属性，这使它们能够表示连续值（type =“linear”，type =“log”），时间值（type =“date”）或分类值（type =“category”），type =“multicategory“）。轴也可以彼此覆盖以创建双轴或多轴图表。2-D cartesian子图非常适合创建“small multples”figure，也称为facet或trellis plots。

以下trace类型与Xaxis和Yaxis属性与2D-Cartesian子图兼容：
* scatter-like trace types:scatter 和 scattergl，可用于绘制散点图，线图和曲线，时间序列图，气泡图，点图和填充区域，还支持错误条
* bar, funnel, waterfall: bar-like trace types ，也可用于绘制时间表和甘特图表
* histogram:直方图
* box 和 violin: 一维分布trace types。
* histogram2d 和 histogram2dcontour:二维分布样密度trace类型
* carpet: 用于建造carpet图的特殊trace类型，因为其他trace可以用作子图
* splom:多维散点图隐式地同时参考许多2-D笛卡尔子图。